In [130]:
from sklearn.metrics import  accuracy_score
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [131]:
train_size = 0.6
test_size = 0.4

In [132]:
data = load_iris()

In [133]:
dataset = data['data']
target = data['target'].reshape(-1,1)

In [134]:
final_data = np.hstack((dataset,target))
np.random.shuffle(final_data)

In [135]:
pd_data = pd.DataFrame(final_data)
pd_data.columns= ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)','target']
pd_data = pd_data[(pd_data['target']==0) | (pd_data['target']==1)]
pd_data.tail(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
144,5.7,3.8,1.7,0.3,0.0
149,4.8,3.0,1.4,0.3,0.0


In [136]:
final_data = pd_data.values

train_data = final_data[:int(len(final_data)*train_size)]
test_data = final_data[:int(len(final_data)*test_size)]

train_data.shape,test_data.shape

((60, 5), (40, 5))

In [137]:
X_train = train_data[:,:-1]
y_train = train_data[:,-1:]


X_test = test_data[:,:-1]
y_test = test_data[:,-1:]


X_train.shape,y_train.shape,X_test.shape,y_test.shape

((60, 4), (60, 1), (40, 4), (40, 1))

In [138]:
def gradient(y,y_pred,x):
    return np.dot(x.T,(y-y_pred))

In [139]:
def new_grad(x):
    return sigmoid(x)(1-sigmoid(x))

In [140]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [141]:
def cost(y,y_pred):
    cost = -np.mean((y*np.log(y_pred))+((1-y)*(np.log(1-y_pred))))
    return cost

In [171]:
dummy_train = np.hstack((np.ones((len(train_data),1)),X_train))
weight = np.zeros((len(dummy_train[0]),1))
lr = 0.001
epoch = 1000


for i in range(epoch):
    y_pred = sigmoid(np.dot(dummy_train,weight))
    co = cost(y_train,y_pred)
    gra = gradient(y_train,y_pred,dummy_train)
    weight += lr*gra
    
    
    if i%10 == 0:
        print('Epoch : ',i,' Loss:',co)

Epoch :  0  Loss: 0.6931471805599454
Epoch :  10  Loss: 0.4841287811935128
Epoch :  20  Loss: 0.36257954067589704
Epoch :  30  Loss: 0.28620763214258393
Epoch :  40  Loss: 0.23503765414403915
Epoch :  50  Loss: 0.19883339484669688
Epoch :  60  Loss: 0.17206293330535785
Epoch :  70  Loss: 0.1515531314737786
Epoch :  80  Loss: 0.13538120896864156
Epoch :  90  Loss: 0.122325052643349
Epoch :  100  Loss: 0.11157546125197305
Epoch :  110  Loss: 0.10257756791883182
Epoch :  120  Loss: 0.09493930787887361
Epoch :  130  Loss: 0.08837637397640545
Epoch :  140  Loss: 0.08267789722195763
Epoch :  150  Loss: 0.07768435687609061
Epoch :  160  Loss: 0.07327295613060186
Epoch :  170  Loss: 0.06934769514048184
Epoch :  180  Loss: 0.06583248058290432
Epoch :  190  Loss: 0.06266624628549251
Epoch :  200  Loss: 0.05979943515893999
Epoch :  210  Loss: 0.057191420967979203
Epoch :  220  Loss: 0.054808590689287215
Epoch :  230  Loss: 0.05262289881794621
Epoch :  240  Loss: 0.050610763921854214
Epoch :  250 

In [172]:
weight

array([[-0.27988151],
       [-0.46309775],
       [-1.84544876],
       [ 2.76387978],
       [ 1.17888333]])

In [173]:
def predict(x,w):
    sig = sigmoid(np.dot(x,w[1:])+w[0])
    sig[sig > 0.5] = 1
    sig[sig < 0.5] = 0
    return sig

In [175]:
accuracy_score(predict(X_test,weight),y_test)

1.0

In [234]:
class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.01,batch_size=None,epoch=10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if i%10 == 0:
                print('Epoch : {}  Loss: {}'.format(i+1,loss))
    def predict(self,test_data):
        result = sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [235]:
logistic = logisticregression(X_train,y_train,epoch=100)

In [236]:
logistic.train()

Epoch : 1  Loss: 0.6931471805599454
Epoch : 11  Loss: 0.2991661807009864
Epoch : 21  Loss: 0.011371657746110207
Epoch : 31  Loss: 0.01057078520870101
Epoch : 41  Loss: 0.009898383671356207
Epoch : 51  Loss: 0.009310556643020974
Epoch : 61  Loss: 0.008792038558256856
Epoch : 71  Loss: 0.0083310746957437
Epoch : 81  Loss: 0.007918443795442355
Epoch : 91  Loss: 0.007546807899728929


In [237]:
logistic.evaluate(X_test,y_test)

1.0